In [1]:
import tkinter as tk
from tkinter import messagebox
import pickle
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
# Load the Keras model for sentiment
sentiment_model_LSTM_WithoutGS = load_model('sentiment_model_LSTM_WithoutGS.keras')

# Load the tokenizer
with open('tokenizer_LSTM_WithoutGS.pkl', 'rb') as file:
    tokenizer_LSTM_WithoutGS = pickle.load(file)

# Load the label encoder for sentiment
with open('label_encoder_sentiment_LSTM_WithoutGS.pkl', 'rb') as f:
    label_encoder_sentiment_LSTM_WithoutGS = pickle.load(f)

# Load the Keras model for capital
capital_model_LSTM_WithoutGS = load_model('capital_model_LSTM_WithoutGS.keras')

# Load the TF-IDF vectorizer
with open('tfidf_vectorizer_LSTM_WithoutGS.pkl', 'rb') as file:
    tfidf_vectorizer_LSTM_WithoutGS = pickle.load(file)

# Load the label encoder for capital
with open('label_encoder_capital_LSTM_WithoutGS.pkl', 'rb') as f:
    label_encoder_capital_LSTM_WithoutGS = pickle.load(f)

# Define max sequence length (must be the same as in the training phase)
max_sequence_length = 5000  # Replace with the actual value used during training


In [3]:
def predict_pipeline(text, max_sequence_length):
    clean_text = text  # Assuming the text is already cleaned
    text_tfidf = tfidf_vectorizer_LSTM_WithoutGS.transform([clean_text])

    # Predict capital
    capital_prediction_encoded = capital_model_LSTM_WithoutGS.predict(text_tfidf).argmax(axis=1)
    capital_prediction = label_encoder_capital_LSTM_WithoutGS.inverse_transform(capital_prediction_encoded)[0]

    # Predict sentiment
    text_seq = tokenizer_LSTM_WithoutGS.texts_to_sequences([clean_text])
    text_pad = pad_sequences(text_seq, maxlen=max_sequence_length)
    sentiment_prediction_encoded = sentiment_model_LSTM_WithoutGS.predict(text_pad).argmax(axis=1)
    sentiment_prediction = label_encoder_sentiment_LSTM_WithoutGS.inverse_transform(sentiment_prediction_encoded)[0]

    return capital_prediction, sentiment_prediction

# Test the prediction function
test_sentence = "Leaders' focus on achieving excellence motivated the team to perform their best. This is a core aspect of quality leadership. 🔝"
capital, sentiment = predict_pipeline(test_sentence, max_sequence_length)
print(f"Predicted Capital: {capital}, Predicted Sentiment: {sentiment}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 801ms/step
Predicted Capital: Human Leadership Capital, Predicted Sentiment: Positive


In [4]:
def predict():
    text = entry.get()
    if not text:
        messagebox.showwarning("Input Error", "Please enter a sentence.")
        return
    predicted_capital, predicted_sentiment = predict_pipeline(text, max_sequence_length)

    result_text = f"Predicted Capital: {predicted_capital}\nPredicted Sentiment: {predicted_sentiment}"
    result_window = tk.Toplevel(root)
    result_window.title("Prediction Result")

    result_label = tk.Label(result_window, text=result_text, font=("Helvetica", 14), padx=20, pady=20)
    result_label.pack()

    close_button = tk.Button(result_window, text="Close", command=result_window.destroy, font=("Helvetica", 12), padx=10, pady=5)
    close_button.pack(pady=10)

root = tk.Tk()
root.title("Capital and Sentiment Prediction")

# Set window size and position
root.geometry("500x250+300+200")

label = tk.Label(root, text="Enter a sentence:", font=("Helvetica", 14))
label.pack(pady=20)

entry = tk.Entry(root, width=50, font=("Helvetica", 12))
entry.pack(pady=10)

predict_button = tk.Button(root, text="Predict", command=predict, font=("Helvetica", 12), padx=10, pady=5)
predict_button.pack(pady=20)

root.mainloop()

# Thank You!